In [ ]:
#setting Output
import pandas as pd
csv_input = pd.read_csv(r'C:\Users\admin\Downloads\QueryResults (1).csv')
csv_input['Output'] = (csv_input['qAcceptedAnswerId']==csv_input['aId'])
csv_input.to_csv(r'C:\Users\admin\Downloads\datasettemp.csv', index=False)

In [ ]:
#Normalising ascore
import pandas as pd
csv_input = pd.read_csv(r'C:\Users\admin\Downloads\datasettemp.csv')
dist = csv_input.groupby('aParentId')['aScore'].transform('max')
dist1=csv_input.groupby('aParentId')['aScore'].transform('min')
dist= dist-dist1
#csv_input.loc[:, csv_input.drop('aScore', axis=1).columns] = (dist.sub(csv_input.drop('aScore', axis=1)).div(dist))
csv_input['anewScore']=(csv_input['aScore']-dist1).div(dist)
csv_input['anewScore'] = csv_input['anewScore'].fillna(0)
csv_input.to_csv(r'C:\Users\admin\Downloads\datasettemp.csv', index=False)

In [ ]:
#Normalising Commentcount
import pandas as pd
csv_input = pd.read_csv(r'C:\Users\admin\Downloads\datasettemp.csv')
dist =  csv_input.groupby('aParentId')['aCommentCount'].transform('max')
dist1=csv_input.groupby('aParentId')['aCommentCount'].transform('min')
dist= dist-dist1
#csv_input.loc[:, csv_input.drop('aScore', axis=1).columns] = (dist.sub(csv_input.drop('aScore', axis=1)).div(dist))
csv_input['anewCommentCount']=(csv_input['aCommentCount']-dist1).div(dist)
csv_input['anewCommentCount'] = csv_input['anewCommentCount'].fillna(0)
csv_input.to_csv(r'C:\Users\admin\Downloads\datasettemp.csv', index=False)

In [ ]:
#setting order of the answer
import pandas as pd
csv_input = pd.read_csv(r'C:\Users\admin\Downloads\datasettemp.csv')
dist =  csv_input.groupby('aParentId')['aCreationDate'].cumcount()+1
csv_input['aorder']=dist
csv_input.to_csv(r'C:\Users\admin\Downloads\datasettemp.csv', index=False)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from bs4 import BeautifulSoup
import pandas as pd
import tokenize
import io
import nltk 
import re
from nltk.stem import WordNetLemmatizer 
nltk.download('averaged_perceptron_tagger') 
from nltk.corpus import wordnet 
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize 

csv_input = pd.read_csv(r'C:\Users\admin\Downloads\datasettemp.csv')

stop_words = set(stopwords.words('english'))

lemmatizer = WordNetLemmatizer() 
     
def get_cosine_sim(strs):
    vectors = [t for t in get_vectors(strs)]
    k=cosine_similarity(vectors)
    return k[0][1]
     


def pos_tagger(nltk_tag): 
    if nltk_tag.startswith('J'): 
        return wordnet.ADJ 
    elif nltk_tag.startswith('V'): 
        return wordnet.VERB 
    elif nltk_tag.startswith('N'): 
        return wordnet.NOUN 
    elif nltk_tag.startswith('R'): 
        return wordnet.ADV 
    else:
        return None


def lemmatize_sentence(sentence): 
    tokens=[w for w in nltk.word_tokenize(sentence)]
    word=[]
    for i in tokens:
        if i not in stop_words:
            word.append(i)
    pos_tagged = nltk.pos_tag(word) 
    wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged)) 
    result = [] 
    for word, tag in wordnet_tagged: 
        if tag is None: 
            continue
        else: 
            result.append(lemmatizer.lemmatize(word, tag)) 
    return result


def get_vectors(strs):
    text = [t for t in strs]
    try:
        vectorizer = CountVectorizer(tokenizer=lemmatize_sentence)
        k=vectorizer.fit_transform(text).toarray()
    except:
         return [[1,1],[1,1]]
    return k
 
def get_code_cosine_sim(strs):
    temp=strs[:]
    n=len(strs)
    #55 - comment
    stops=[55,4,0]
    d=dict()
    k=[]
    try:
        for i in temp:
             for token in tokenize.generate_tokens(io.StringIO(i).readline):
                if token.type not in stops and not(token.type== 3 and (token.string.startswith('"""') or token.string.startswith("'''"))):
                    space=token.string.replace(' ','')
                    if(len(space)==0):
                        continue
                    index=token.string.find('\n')
                    index1=token.string.find('\r')
                    if(index==-1 and index1==-1):
                        k.append(token.string)
                    elif(index>0 and index1==-1):
                        if(token.string[index-1]=='\''):
                            k.append(token.string)
                    elif(index1==-1 and index1>0):
                        if(token.string[index1-1]=='\''):
                            k.append(token.string)
                    elif(index1>0 and index>0):
                        k.append(token.string)
                        
        for j in k:
            d[j]=[0]*n
        count=0
        for i in temp:
            for token in tokenize.generate_tokens(io.StringIO(i).readline):
                if token.string in k:
                    d[token.string][count]+=1
            count+=1
        a=[]
        b=[]
        for key, val in d.items():
                 a.append(val[0])
                 b.append(val[1])
        vectors=[]
        vectors.append(a)
        vectors.append(b)
        try:
            k=cosine_similarity(vectors)
            return k[0][1]
        except:
            #no code segment
            return 1
    except:
        vectorizer = CountVectorizer(tokenizer=word_tokenize)
        vectors=vectorizer.fit_transform(temp).toarray()
        k=cosine_similarity(vectors)
        return k[0][1]
        
    

In [ ]:

l=[]
for i,j in zip(csv_input['qBody'], csv_input['aBody']):
    soup = BeautifulSoup(i,features="lxml")
    ans=""
    ans1=""
    soup1 = BeautifulSoup(j,features="lxml")
    que=""
    que1=""
    qsim=0
    csim=0

    c_qtags=soup.findAll('pre')
    remove1=soup.findAll('a')
    c_atags=soup1.findAll('pre')
    remove2=soup1.findAll('a')
    for node in c_qtags:
        que1+=node.get_text()
        node.extract()
    for node in remove1:
        node.extract()
    que=soup.get_text()
    for node in c_atags:
        ans1+=node.get_text()
        node.extract()
    for node in remove2:
        node.extract()
    ans=soup1.get_text()
    qsim=get_cosine_sim([que,ans])
    csim=get_code_cosine_sim([que1,ans1])
    l.append([qsim,csim ])

df = pd.DataFrame(l, columns = ['QAsim','QAcodesim'])
csv_input['QAsim']=df['QAsim']
csv_input['QAcodesim ']=df['QAcodesim']

csv_input.to_csv(r'C:\Users\admin\Downloads\datasettemp.csv')

In [ ]:
import pandas as pd
csv_input = pd.read_csv(r'C:\Users\admin\Downloads\datasettemp.csv')
dist =  csv_input.groupby('aParentId')
simt=[]
simc=[]
f=0
for i in dist.aBody.unique():
    text=[]
    code=[]
    anst=""
    ansc=""
    for j in i:
        f+=1
        j=str(j)
        soup = BeautifulSoup(j,"lxml")
        to_remove = soup.find_all("pre") 
        to_remove1=soup.find_all("a")
        for node in to_remove:
            ansc+=node.get_text()
            node.extract()
        for node in to_remove1:
            node.extract()
        anst=soup.get_text()
        text.append(anst)
        code.append(ansc)
    print(ansc)
    print(anst)
    for j in range(0,len(code)):
        sum1=0
        sum2=0
        for k in range(0,len(code)):
            if(j!=k and (text[j]!='' or text[k]!='')):
                sum1+=get_cosine_sim([text[j],text[k]])
            elif j!=k:
                sum1+=1
            if(j!=k and (code[j]!="" or code[k]!="")):
                sum2+=get_code_cosine_sim([code[j],code[k]])
            elif j!=k:
                sum2+=1
        simt.append(sum1/(len(code)-1))
        simc.append(sum2/(len(code)-1))

In [ ]:
df = pd.DataFrame(simt, columns = ['AAsim'])
csv_input['AAsim']=df['AAsim']
df = pd.DataFrame(simc, columns = ['AAcodesim'])
csv_input['AAcodesim']=df['AAcodesim']
csv_input.to_csv(r'C:\Users\admin\Downloads\datasettemp.csv', index=False)  

In [ ]:
import pandas as pd
csv_input = pd.read_csv(r'C:\Users\admin\Downloads\datasettemp.csv')
dist =  csv_input.groupby('aParentId')['QAsim'].transform('max')
dist2 =  csv_input.groupby('aParentId')['QAsim'].transform('min')
dist1=  csv_input.groupby('aParentId')['QAcodesim '].transform('max')
dist3=  csv_input.groupby('aParentId')['QAcodesim '].transform('min')
dist=dist-dist2
dist1=dist1-dist3
csv_input['QAnorm']=(csv_input['QAsim']-dist2).div(dist)
csv_input['QAnorm'] = csv_input['QAnorm'].fillna(0)
csv_input['QAcnorm']=(csv_input['QAcodesim ']-dist3).div(dist1)
csv_input['QAcnorm'] = csv_input['QAcnorm'].fillna(0)
csv_input.to_csv(r'C:\Users\admin\Downloads\datasettemp.csv', index=False)

In [ ]:
import pandas as pd
csv_input = pd.read_csv(r'C:\Users\admin\Downloads\datasettemp.csv')
dist =  csv_input.groupby('aParentId')['AAsim'].transform('max')
dist2 =  csv_input.groupby('aParentId')['AAsim'].transform('min')
dist1=  csv_input.groupby('aParentId')['AAcodesim'].transform('max')
dist3=  csv_input.groupby('aParentId')['AAcodesim'].transform('min')
dist=dist-dist2
dist1=dist1-dist3
csv_input['AAnorm']=(csv_input['AAsim']-dist2).div(dist)
csv_input['AAnorm'] = csv_input['AAnorm'].fillna(0)
csv_input['AAcnorm']=(csv_input['AAcodesim']-dist3).div(dist1)
csv_input['AAcnorm'] = csv_input['AAcnorm'].fillna(0)
csv_input.to_csv(r'C:\Users\admin\Downloads\datasettemp.csv', index=False)

In [3]:
import pandas as pd
df = pd.read_csv(r'C:\Users\admin\Downloads\datasettemp.csv')
x1 = df.loc[df['Output']==0]
x2 = df.loc[df['Output']==1]
x1=x1.sample(frac=1)
X_false=x1.iloc[:4013,:]
X_true=x2.iloc[:,:]
X = pd.concat([X_false, X_true])
X.to_csv(r'C:\Users\admin\Desktop\fypnew.csv')